In [5]:
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer

In [6]:
HF_token = userdata.get("HF_TOKEN")
login( HF_token, add_to_git_credential = True)

In [9]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.3-8B', trust_remote_code=True)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B.
403 Client Error. (Request ID: Root=1-676e3116-27ada3d15fb2b8ff294eba79;285c3981-9ab6-4768-8aec-15caf346dc14)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B/resolve/main/config.json.
Your request to access model meta-llama/Llama-3.1-8B has been rejected by the repo's authors.

In [11]:
PHI3_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
QWEN2_MODEL_NAME = "Qwen/Qwen2-7B-Instruct"
STARCODER2_MODEL_NAME = "bigcode/starcoder2-3b"

In [14]:
text ="I am excited to show Tokenizers in action to my LLM engineers"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

In [13]:
pi_tokeniser = AutoTokenizer.from_pretrained (PHI3_MODEL_NAME)
print(pi_tokeniser.encode (text))

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

[306, 626, 24173, 304, 1510, 25159, 19427, 297, 3158, 304, 590, 365, 26369, 6012, 414]


In [15]:
pi_tokeniser.apply_chat_template(messages,tokenize=False, add_genertion_promplt=True )

'<|system|>\nYou are a helpful assistant<|end|>\n<|user|>\nTell a light-hearted joke for a room of Data Scientists<|end|>\n<|endoftext|>'